In [3]:
import pandas as pd
import numpy as np
from jupyter_utils import jupyter_setup
jupyter_setup()

C:\Michi\acoustic_covid_detection\python


In [32]:
all_types_of_recording = ["cough-heavy", "cough-shallow", "breathing-deep", "breathing-shallow", "counting-fast",
                          "counting-normal", "vowel-a", "vowel-e", "vowel-o"]

qual_names = ["audio_quality_deep_breathing", "audio_quality_shallow_breathing", "audio_quality_heavy_cough", 
"audio_quality_shallow_cough", "audio_quality_counting_fast", "audio_quality_counting_normal", 
"audio_quality_vowel_a", "audio_quality_vowel_e", "audio_quality_vowel_o"]

In [7]:
df = pd.read_csv("data/Coswara_processed/audio_quality_annotations_df.csv")
df_qual = df.drop("user_id", axis=1)
df_qual.head(3)

,audio_quality_deep_breathing,audio_quality_shallow_breathing,audio_quality_heavy_cough,audio_quality_shallow_cough,audio_quality_counting_fast,audio_quality_counting_normal,audio_quality_vowel_a,audio_quality_vowel_e,audio_quality_vowel_o
0,2,2,2.0,2.0,2.0,2.0,2.0,2,2.0
1,0,0,NaN,NaN,NaN,NaN,NaN,1,NaN
2,0,1,2.0,1.0,2.0,2.0,2.0,2,2.0


# valid labels per type of recording

In [11]:
np.sum((df_qual > 0))

audio_quality_deep_breathing       2294
audio_quality_shallow_breathing    2274
audio_quality_heavy_cough          2094
audio_quality_shallow_cough        2096
audio_quality_counting_fast        2055
audio_quality_counting_normal      2063
audio_quality_vowel_a              2050
audio_quality_vowel_e              2485
audio_quality_vowel_o              1971
dtype: int64

# IDs that are valid for EVERY type of recording

In [21]:
def get_valid_rows(data):
    data = pd.DataFrame(data > 0)
    rows = np.mean(data, axis=1)
    rows = rows == 1
    return rows

In [23]:
get_valid_rows(df_qual).sum()  # all 9 recording types

1564

In [43]:
drops = ["audio_quality_deep_breathing", "audio_quality_shallow_breathing", "audio_quality_heavy_cough" ,
         "audio_quality_shallow_cough"]
drops = ["audio_quality_shallow_breathing", "audio_quality_shallow_cough", "audio_quality_counting_fast", "audio_quality_counting_normal", "audio_quality_vowel_e", "audio_quality_vowel_o"]
df2 = pd.DataFrame(df_qual)
for drop in drops:
    #drop = f"audio_quality_{drop}"
    df2 = df2.drop(drop, axis=1)
df2.head(2)

,audio_quality_deep_breathing,audio_quality_heavy_cough,audio_quality_vowel_a
0,2,2.0,2.0
1,0,NaN,NaN


In [44]:
get_valid_rows(df2).sum() 

1835

In [77]:
def combine_two_cols(df, cols, name):
    new_df = pd.DataFrame(df)
    col0 = df[cols[0]]
    col1 = df[cols[1]]
    new_df = new_df.drop(cols[0], axis=1)
    new_df = new_df.drop(cols[1], axis=1)
    combined = np.logical_or(col0, col1)
    combined = pd.DataFrame({name: combined})
    new_df = pd.concat([new_df, combined], axis=1)
    return new_df
    


In [93]:
df_bool = df_qual > 0
cols = ["audio_quality_deep_breathing", "audio_quality_shallow_breathing"]
df_bool_new = combine_two_cols(df_bool, cols, "breath")
cols = ["audio_quality_heavy_cough",
         "audio_quality_shallow_cough"]
df_bool_new = combine_two_cols(df_bool_new, cols, "cough")
df_bool_new.head(3)

,audio_quality_counting_fast,audio_quality_counting_normal,audio_quality_vowel_a,audio_quality_vowel_e,audio_quality_vowel_o,breath,cough
0,True,True,True,True,True,True,True
1,False,False,False,True,False,False,False
2,True,True,True,True,True,True,True


In [106]:
drops = ["audio_quality_deep_breathing", "audio_quality_shallow_breathing", "audio_quality_heavy_cough" ,
         "audio_quality_shallow_cough"]
drops = ["audio_quality_counting_fast", 
         "audio_quality_counting_normal", 
         "audio_quality_vowel_e", 
         "audio_quality_vowel_o", 
        # "audio_quality_vowel_a"
        ]
df2 = pd.DataFrame(df_bool_new)
for drop in drops:
    #drop = f"audio_quality_{drop}"
    df2 = df2.drop(drop, axis=1)
df2.head(2)

,audio_quality_vowel_a,breath,cough
0,True,True,True
1,False,False,False


In [107]:
get_valid_rows(df2).sum() 

1928